In [1]:
import pandas as pd
import requests
import json
import datetime

In [2]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

In [3]:
req = requests.get('https://api.sncf.com/v1/coverage/sncf/vehicle_journeys/vehicle_journey:SNCF:2023-03-02:9713:1187:LongDistanceTrain',auth=(token, ''))

In [4]:
doc = json.loads(req.text)
row = len(doc['disruptions'])
print(f'Nombre de lignes : {row}')

Nombre de lignes : 1


# Retard

## Liste des retards

In [5]:
df = pd.DataFrame(doc['disruptions'])
df.head()

,status,disruption_id,severity,impact_id,application_periods,messages,updated_at,uri,impacted_objects,disruption_uri,contributor,cause,id
0,active,49b3f803-d5a1-4fb5-a17f-4f15e6fab2c3,"{'color': '#000000', 'priority': 42, 'name': '...",49b3f803-d5a1-4fb5-a17f-4f15e6fab2c3,"[{'begin': '20230302T093800', 'end': '20230302...",[{'text': 'Intervention des forces de l'ordre'...,20230302T111443,49b3f803-d5a1-4fb5-a17f-4f15e6fab2c3,[{'impacted_stops': [{'amended_arrival_time': ...,49b3f803-d5a1-4fb5-a17f-4f15e6fab2c3,realtime.sncf.piv,,49b3f803-d5a1-4fb5-a17f-4f15e6fab2c3


## Etat du retard

In [13]:
df_info = pd.DataFrame(list(df['messages'][0]))
df_info.head()

,text,channel
0,Intervention des forces de l'ordre,"{'content_type': '', 'id': 'rt', 'types': ['we..."


In [14]:
df_info.iloc[0]['text']

"Intervention des forces de l'ordre"

## Combinaison

In [7]:
df_fin = df_info.copy()
df_fin = df_fin[df_fin['effect'] == 'SIGNIFICANT_DELAYS']
df_fin = df_fin[['effect']]
df_fin

,effect
0,SIGNIFICANT_DELAYS


In [8]:
gare_d = []
gare_a = []
cause = []
retard = []
heure = []

for i in df_fin.index:
    df_mod = pd.DataFrame(list(df['impacted_objects'][i]))
    df_mod = pd.DataFrame(list(df_mod['impacted_stops'])[0])
    df_mod = df_mod[(df_mod['amended_arrival_time'].notnull())]
    df_mod = df_mod[(df_mod['base_arrival_time'].notnull())]

    retard_val = int(df_mod.iloc[-1]['amended_arrival_time']) - int(df_mod.iloc[-1]['base_arrival_time'])

    df_mod['gare'] = pd.DataFrame(list(df_mod['stop_point']))['name']

    gare_d.append(df_mod.iloc[0]['gare'])
    gare_a.append(df_mod.iloc[-1]['gare'])
    cause.append(df_mod.iloc[-1]['cause'])
    retard.append(str(retard_val)[:-2])
    heure.append(df_mod.iloc[0]['amended_departure_time'])

In [9]:
df_fin['gare_depart'] = gare_d
df_fin['gare_arrivee'] = gare_a
df_fin['cause'] = cause
df_fin['retard_min'] = retard
df_fin['heure_depart'] = heure

In [10]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_fin['heure_depart'] = df_fin['heure_depart'].apply(str_tps)

In [11]:
df_fin

,effect,gare_depart,gare_arrivee,cause,retard_min,heure_depart
0,SIGNIFICANT_DELAYS,Paris - Gare de Lyon - Hall 1 & 2,Barcelona Sants,Intervention des forces de l'ordre,85,10:23:00
